### Open file and prepare Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("515k-hotel-reviews-data-in-europe.zip")

In [3]:
def get_country(adress):
    country = adress.split()[-1]
    if country == "Kingdom":
        return("United Kingdom")
    else:
        return(country)
    
df['Country'] = df.Hotel_Address.apply(lambda x: get_country(x))
df.Country.value_counts()

United Kingdom    262301
Spain              60149
France             59928
Netherlands        57214
Austria            38939
Italy              37207
Name: Country, dtype: int64

In [4]:
def get_city(adress, country):
    city = adress.split()[-2]
    if country == "United Kingdom":
        return(adress.split()[-5])
    else:
        return(city)

df['City'] = df[['Hotel_Address','Country']].apply(lambda x: get_city(x[0], x[1]), axis=1)
df.City.value_counts()

London       262301
Barcelona     60149
Paris         59928
Amsterdam     57214
Vienna        38939
Milan         37207
Name: City, dtype: int64

### Scrapping Hotel Prices
There's a lot more of information missing which we think is important to get better insights. 
To get all this information the first thing i need is to get the exact address that directs me into the booking page containing information of each Hotel. 
This first step will be done through scrapping the links in Google.
Once scrapped the final address in booking, another scrapping exercise will be needed in order to get the desired information for each Hotel.

In [76]:
hotel_city = df[['Hotel_Name','City']].groupby(['Hotel_Name','City']).count().reset_index()
scrap = hotel_city.apply(lambda x: ('Hotel ' + x[0] + ' ' + x[1]), axis=1)
scrap.head()

0                    Hotel 11 Cadogan Gardens London
1                               Hotel 1K Hotel Paris
2    Hotel 25hours Hotel beim MuseumsQuartier Vienna
3                                    Hotel 41 London
4    Hotel 45 Park Lane Dorchester Collection London
dtype: object

This is the adress i will use in Google. The idea is that i will pick the first adress coming from booking.com as the final address of the hotel in the sistem. Once i will gather all of the addresses i will try to scrap the prices in each one of them

In [79]:
google_address = ['https://www.google.com/search?sxsrf=&q=booking.com+'+i.replace(" ", "+") for i in scrap]

In [81]:
google_address[:5]

['https://www.google.com/search?sxsrf=&q=booking.com+Hotel+11+Cadogan+Gardens+London',
 'https://www.google.com/search?sxsrf=&q=booking.com+Hotel+1K+Hotel+Paris',
 'https://www.google.com/search?sxsrf=&q=booking.com+Hotel+25hours+Hotel+beim+MuseumsQuartier+Vienna',
 'https://www.google.com/search?sxsrf=&q=booking.com+Hotel+41+London',
 'https://www.google.com/search?sxsrf=&q=booking.com+Hotel+45+Park+Lane+Dorchester+Collection+London']

Once i've got all the addresses of a Google search i start scrapping the addresses in Booking.

I also save the important information just in case i need to start over again

In [8]:
import pickle
pickle.dump(hotel_city, open('./hotel_city.sav', 'wb'))
pickle.dump(google_address, open('./google_address.sav', 'wb'))

#### Scrapping of Booking addresses

In [9]:
import requests
from bs4 import BeautifulSoup
import time

In [10]:
bookingaddress = []

for i, hotel in enumerate(google_address[:5]):
    URL = hotel
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    bkng = []
    for page in soup.find_all('a', href=True):
        if page['href'][:37] == '/url?q=https://www.booking.com/hotel/':
            bkng.append(page['href'][7:].split('&', 1)[0])
    if bkng[0][-8:] != '.es.html':
        bkng[0] = bkng[0][:-5]+'.es.html'
        
    bookingaddress.append(bkng[0])
    print(i,bkng[0])
    time.sleep(3)

0 https://www.booking.com/hotel/gb/number-eleven.es.html
1 https://www.booking.com/hotel/fr/1-k-hotel.es.html
2 https://www.booking.com/hotel/at/25hours-wien.es.html
3 https://www.booking.com/hotel/gb/41clubredcarnations.es.html
4 https://www.booking.com/hotel/gb/parklane.es.html


#### Scrapping of other Booking information (sample of 1)

In [11]:
import json
i = 0

In [12]:
head = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36"}
URL = bookingaddress[i]
page = requests.get(URL, headers=head)
soup = BeautifulSoup(page.content, 'html.parser')
print(bookingaddress[i])

https://www.booking.com/hotel/gb/number-eleven.es.html


Get the Header of the page ¿?

In [13]:
data = soup.select("[type='application/ld+json']")[0]

Get the price from the Header

In [19]:
pricetag = json.loads(data.text)['priceRange']
price = int(pricetag.split("€",1)[1].split()[0])

Get the stars of the Hotel

In [21]:
stars = soup.find(class_='star_track')['title']

Get the name of the Hotel

In [20]:
name = json.loads(data.text)['name']

Get the facilities from Booking

In [26]:
facilities = {}
raw = soup.find_all(class_='facilitiesChecklistSection')
for i in raw:
    dept = i.getText().split('\n')
    facilities_dept = [x for x in i.getText().split('\n') if x]
    dept = facilities_dept[0]
    items = facilities_dept[1:]    
    facilities[dept] = items

Create the Dictionary with the data

In [50]:
hotels = {}
hotels[name] = {'Price': price, 'Stars': stars, 'Facilities': facilities, 'Header': json.loads(data.text)}

#### Scrapping of other Booking information (complete dataset for loop)

In [62]:
import json

In [63]:
head = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36"}
hotels = {}
name_scrap = []

for num, url in enumerate(bookingaddress):
    
    print(num, url)
    
    URL = url
    page = requests.get(URL, headers=head)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get Header
    data = soup.select("[type='application/ld+json']")[0]
    
    # Get Hotel Name
    name = json.loads(data.text)['name']
    name_scrap.append(name)
    
    # Get Price
    pricetag = json.loads(data.text)['priceRange']
    price = int(pricetag.split("€",1)[1].split()[0])
    
    # Get Stars
    stars = soup.find(class_='star_track')['title']
    
    # Get Facilities    
    facilities = {}
    raw = soup.find_all(class_='facilitiesChecklistSection')
    for i in raw:
        dept = i.getText().split('\n')
        facilities_dept = [x for x in i.getText().split('\n') if x]
        dept = facilities_dept[0]
        items = facilities_dept[1:]    
        facilities[dept] = items
        
    # Append Information to Dicctionary
    hotels[name] = {'Price': price, 'Stars': stars, 'Facilities': facilities, 'Header': json.loads(data.text)}

0 https://www.booking.com/hotel/gb/number-eleven.es.html
1 https://www.booking.com/hotel/fr/1-k-hotel.es.html
2 https://www.booking.com/hotel/at/25hours-wien.es.html
3 https://www.booking.com/hotel/gb/41clubredcarnations.es.html
4 https://www.booking.com/hotel/gb/parklane.es.html
5 https://www.booking.com/hotel/gb/88-studios.es.html
6 https://www.booking.com/hotel/fr/9hotel-republique.es.html
7 https://www.booking.com/hotel/fr/a-la-villa-madame.es.html
8 https://www.booking.com/hotel/es/abac-barcelona.es.html
9 https://www.booking.com/hotel/es/achotelsbarcelona.es.html
10 https://www.booking.com/hotel/es/ac-marriott-diagonal-lilla.es.html
11 https://www.booking.com/hotel/es/acirla.es.html
12 https://www.booking.com/hotel/it/ac-milano.es.html
13 https://www.booking.com/hotel/fr/ac-paris-porte-maillot-by-marriott.es.html
14 https://www.booking.com/hotel/es/ac-sants.es.html


In [25]:
#name_scrap

In [22]:
#hotels

In [42]:
#[i for i in hotels]

[nan, 'The Adria Hotel']

In [24]:
#for i in hotels:
#    print(hotels[i]['Header']['url'])

#### All the process in only one For Loop

In [70]:
import requests
from bs4 import BeautifulSoup
import time

In [71]:
hotel_city = pickle.load(open('./hotel_city.sav', 'rb'))
google_address = pickle.load(open('./google_address.sav', 'rb'))

In [72]:
head = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36"}
bookingaddress = []
hotels = {}
name_scraped = []
address_scraped = []
begin = 0

In [73]:
for i, hotel in enumerate(google_address[begin:]):
    
    # Get Booking.com Address from Google
    URL = hotel
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    bkng = []
    for page in soup.find_all('a', href=True):
        if page['href'][:37] == '/url?q=https://www.booking.com/hotel/':
            bkng.append(page['href'][7:].split('&', 1)[0])
    if bkng[0][-8:] != '.es.html':
        bkng[0] = bkng[0][:-5]+'.es.html'
        
    bookingaddress.append(bkng[0])
    print(i+begin,bkng[0])
    time.sleep(6)
        
    # Open Booking.com webpage
    URL = bkng[0]
    page = requests.get(URL, headers=head)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get Header
    if len(soup.select("[type='application/ld+json']")) > 0:
        data = soup.select("[type='application/ld+json']")[0]
        hotel_header = json.loads(data.text)
    else:
        continue
    
    # Get Hotel Name
    name = json.loads(data.text)['name']
    name_scraped.append(name)
    address_scraped.append(hotel)
    
    # Get Price
    pricetag = json.loads(data.text)['priceRange']
    if price != None:
        price = int(pricetag.split("€",1)[1].split()[0])
    else:
        price = np.nan
        
    # Get Stars
    if soup.find(class_='star_track') != None: 
        stars = soup.find(class_='star_track')['title']
    else:
        stars = np.nan
    
    # Get Facilities    
    facilities = {}
    raw = soup.find_all(class_='facilitiesChecklistSection')
    for i in raw:
        dept = i.getText().split('\n')
        facilities_dept = [x for x in i.getText().split('\n') if x]
        dept = facilities_dept[0]
        items = facilities_dept[1:]    
        facilities[dept] = items
        
    # Append Information to Dicctionary
    hotels[name] = {'Price': price, 'Stars': stars, 'Facilities': facilities, 'Header': hotel_header}

0 https://www.booking.com/hotel/gb/number-eleven.es.html
1 https://www.booking.com/hotel/fr/1-k-hotel.es.html
2 https://www.booking.com/hotel/at/25hours-wien.es.html
3 https://www.booking.com/hotel/gb/41clubredcarnations.es.html
4 https://www.booking.com/hotel/gb/parklane.es.html
5 https://www.booking.com/hotel/gb/88-studios.es.html
6 https://www.booking.com/hotel/fr/9hotel-republique.es.html
7 https://www.booking.com/hotel/fr/a-la-villa-madame.es.html
8 https://www.booking.com/hotel/es/abac-barcelona.es.html
9 https://www.booking.com/hotel/es/achotelsbarcelona.es.html
10 https://www.booking.com/hotel/es/ac-marriott-diagonal-lilla.es.html
11 https://www.booking.com/hotel/es/acirla.es.html


IndexError: list index out of range

In [ ]:
import pickle
pickle.dump(hotels, open('./hotels.sav', 'wb'))
pickle.dump(bookingaddress, open('./bookingaddress.sav', 'wb'))
pickle.dump(name_scrap, open('./name_scrap.sav', 'wb'))